In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

# Load model
model = load_model("soil_classifier_resnet50.h5")

# Load test image IDs
test_df = pd.read_csv("/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv")
test_image_dir = "/kaggle/input/soil-classification-part-2/soil_competition-2025/test"

# Preprocess test images
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_image_dir,
    x_col='image_id',
    y_col=None,
    target_size=(224, 224),
    class_mode=None,
    batch_size=1,
    shuffle=False
)

# Predict probabilities
pred_probs = model.predict(test_gen, verbose=1)

# Convert to binary predictions (threshold = 0.5)
pred_labels = (pred_probs > 0.5).astype(int).flatten()

# Create submission
submission_df = pd.DataFrame({
    'image_id': test_df['image_id'],
    'label': pred_labels
})

# Save submission
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved to submission.csv")